In [1]:
import os
import torch
import random
import numpy as np
import pandas as pd
from datasets import load_dataset
import nlpaug.augmenter.sentence as nas
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
from transformers import MarianMTModel, MarianTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
aug = nas.AbstSummAug(model_path='t5-large', tokenizer_path='t5-large', max_length=185, batch_size=16, device=device)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:161: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

In [3]:
def get_dataset(dataset_name):
    if dataset_name == 'covid':
        # Load the dataset into a pandas dataframe.
        dataset = load_dataset('llangnickel/long-covid-classification-data')
        # get the training data
        batch = dataset['train']['text']
        labels = dataset['train']['label']
        
    elif dataset_name == 'cancer':
        classes = {'Thyroid_Cancer' : 0,  'Lung_Cancer' : 1,  'Colon_Cancer' : 2}
        # Load the dataset into a pandas dataframe.
        df = pd.read_csv('../../cancer.csv', encoding='latin')
        values = df.values

        # get the entire data
        all_batch  = list(values[:,2])
        str_labels = list(values[:,1])
        all_labels = [classes[k] for k in str_labels]

        # training and test data split
        batch, _, labels, _ = train_test_split(all_batch, all_labels, test_size=0.5, random_state=42)
        batch = batch[:100] # since it is an easy dataset use just 500 data points
        labels = labels[:100]
    
    elif dataset_name == 'medical_texts':
        
        f = open('../../Medical texts/train.dat', 'r')
        lines = f.readlines()
        batch = list()
        labels = list()
        for line in lines:
            labels.append(int(line[0])-1) # subtract 1 to make it in the range required by the model
            batch.append(line[2:len(line)-1])
        f.close()

        # training and test data split
        batch, test_batch, labels, test_labels = train_test_split(batch, labels, test_size=0.2, random_state=42)
        
        batch = batch[:500] # since it is an easy dataset use just 500 data points
        labels = labels[:500]
            
    elif dataset_name == 'medical_trans':
        
        label_dict = {' Neurology': 0,
                     ' Discharge Summary': 1,
                     ' Psychiatry / Psychology': 2,
                     ' Pediatrics - Neonatal': 3,
                     ' Neurosurgery': 4,
                     ' Gastroenterology': 5,
                     ' Emergency Room Reports': 6,
                     ' Ophthalmology': 7,
                     ' Office Notes': 8,
                     ' Cardiovascular / Pulmonary': 9,
                     ' Nephrology': 10,
                     ' ENT - Otolaryngology': 11,
                     ' Hematology - Oncology': 12,
                     ' Obstetrics / Gynecology': 13,
                     ' Orthopedic': 14,
                     ' Urology': 15,
                     ' Radiology': 16,
                     ' General Medicine': 17,
                     ' Pain Management': 18,
                     ' SOAP / Chart / Progress Notes': 19,
                     ' Consult - History and Phy.': 20,
                     ' Surgery': 21}
        
        f = open('medical_transcriptions.txt', 'r')
        lines = f.readlines()
        # get the training data
        all_batch = []
        all_labels = []
        for line in lines:
            parts = line.split('\t')
            batch_label = label_dict[parts[0]]
            if batch_label != 20 and batch_label != 21: # if included these labels there will be huge data imbalance
                all_labels.append(batch_label)
                all_batch.append(parts[1])
        f.close()
        
        # training and test data split
        batch, test_batch, labels, test_labels = train_test_split(all_batch, all_labels, test_size=0.835, random_state=42)
        
    return batch, labels

In [4]:
dataset = 'medical_trans'
original_texts, original_labels = get_dataset(dataset)

In [5]:
len(original_texts), len(original_labels)

(500, 500)

In [6]:
truncated_texts = []
for i in range(len(original_texts)):
    truncated_texts.append(' '.join(original_texts[i].split()[:256]))

In [7]:
augmented_data = []

for i in range(len(truncated_texts)):
    augmented_data.append(aug.augment(truncated_texts[i]))

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


In [8]:
f = open("Data/abstractivesummarization_"+str(dataset)+".txt" , "a" )

for i in range(len(augmented_data)):
    f.write(str(original_labels[i]) + '\t' + augmented_data[i][0] + '\n')
    
f.close()

In [9]:
augmented_data[10][0]

'hexamethylene bisacetamide is a potent polar-planar differentiating agent of leukemia and solid tumor cell lines in vitro at clinically achievable concentrations. HMBA produced nearly identical inhibition of normal and myelodysplastic hematopoietic progenitors.'

In [10]:
original_labels[10]

4